In [3]:
import pandas as pd
import pathlib
from pathlib import Path
import numpy as np
import csv

In [4]:
def convert_14c_to_standard_format(input_path, output_path):
    # Read the first line to detect delimiter
    with open(input_path, 'r') as f:
        first_line = f.readline()
        delimiter = ','

    # Load file with detected delimiter, skip comments
    df = pd.read_csv(
        input_path,
        comment="#",
        sep=delimiter,
        header=None,
        names=["Cal_BP", "14C_age", "Sigma1", "Delta_14C", "Sigma2"]
    )

    # Rename and keep only the relevant columns
    df = df.rename(columns={
        "Cal_BP": "calendar_year",
        "14C_age": "carbon_year",
        "Sigma2": "carbon_error"
    })
    df_out = df[["calendar_year", "carbon_year", "carbon_error"]].copy()

    # Convert to numeric (cleaning strings)
    df_out = df_out.apply(pd.to_numeric, errors="coerce")

    # Drop rows with bad data
    df_out = df_out.dropna()

    # Sort by calendar year
    df_out = df_out.sort_values("calendar_year").reset_index(drop=True)

    # Add index column
    df_out.insert(0, "", df_out.index)

    # Write clean CSV output with desired header
    with open(output_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["", "Calendar_year", "Carbon_year", "Carbon_error"])
        writer.writerows(df_out.values)

In [7]:
input_path = Path("marine20.14c")
output_path = input_path.with_name(input_path.stem + "_converted.csv")

convert_14c_to_standard_format(input_path, output_path)

In [12]:
# Load the original CSV file
file_path = "marine20_converted.csv"
df = pd.read_csv(file_path)

# Define the new range of calendar years
new_years = np.arange(0, 55001)

# Perform linear interpolation for 'Carbon_year' and 'Carbon_error'
carbon_interp = np.interp(new_years, df['Calendar_year'], df['Carbon_year'])
error_interp = np.interp(new_years, df['Calendar_year'], df['Carbon_error'])

# Create a new DataFrame with the interpolated values
interpolated_df = pd.DataFrame({
    'Calendar_year': new_years,
    'Carbon_year': carbon_interp,
    'Carbon_error': error_interp
})

# Add the index column like the original ('Unnamed: 0' renamed to 'Index')
interpolated_df.insert(0, 'Index', range(len(interpolated_df)))

# Save the interpolated data to a new CSV file
output_path = "marine20_interpolated.csv"
interpolated_df.to_csv(output_path, index=False)